<a href="https://colab.research.google.com/github/Singhrahul2511/NLP_All_Experiment/blob/main/wrd_emb_TF_IDF_BOW_N_grm(ex_6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import re
import string
import pandas as pd
import math
from functools import reduce
from math import log

corpus = """
The quick brown fox jumps over the lazy dog
Never jump over the lazy dog quickly
A lazy dog never jumps over a quick brown fox





""".split("\n")[1:-1]
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)


['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
['never', 'jump', 'over', 'the', 'lazy', 'dog', 'quickly']
['a', 'lazy', 'dog', 'never', 'jumps', 'over', 'a', 'quick', 'brown', 'fox']


In [14]:
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)


{'a', 'jumps', 'never', 'fox', 'brown', 'over', 'lazy', 'the', 'quick', 'jump', 'quickly', 'dog'}


In [17]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:

  word_dict_A[word] += 1


for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])


,a,jumps,never,fox,brown,over,lazy,the,quick,jump,quickly,dog
0,0,1,0,1,1,1,1,2,1,0,0,1
1,0,0,1,0,0,1,1,1,0,1,1,1
2,2,1,1,1,1,1,1,0,1,0,0,1


In [18]:
#compute TF
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

pd.DataFrame([tf_A, tf_B, tf_C])


,a,jumps,never,fox,brown,over,lazy,the,quick,jump,quickly,dog
0,0.0,0.111111,0.000000,0.111111,0.111111,0.111111,0.111111,0.222222,0.111111,0.000000,0.000000,0.111111
1,0.0,0.000000,0.142857,0.000000,0.000000,0.142857,0.142857,0.142857,0.000000,0.142857,0.142857,0.142857
2,0.2,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.000000,0.100000,0.000000,0.000000,0.100000


In [19]:
#compute IDF
def compute_idf(strings_list):
    n = len(strings_list)  # Total number of documents
    idf = dict.fromkeys(strings_list[0].keys(), 0)  # Initialize IDF dictionary

    # Calculate the document frequency (DF) for each word
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1

    # Calculate the IDF for each word using log base 2
    for word, df in idf.items():
        if df > 0:
            idf[word] = math.log2(n / float(df))
        else:
            idf[word] = 0  # Handle cases where the word is not in any document

    return idf

idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])
pd.DataFrame([idf])


,a,jumps,never,fox,brown,over,lazy,the,quick,jump,quickly,dog
0,1.584963,0.584963,0.584963,0.584963,0.584963,0.0,0.0,0.584963,0.584963,1.584963,1.584963,0.0


In [20]:
#compute TF-IDF ratio
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

#print the TF-IDF ratio
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])


,a,jumps,never,fox,brown,over,lazy,the,quick,jump,quickly,dog
0,0.000000,0.064996,0.000000,0.064996,0.064996,0.0,0.0,0.129992,0.064996,0.000000,0.000000,0.0
1,0.000000,0.000000,0.083566,0.000000,0.000000,0.0,0.0,0.083566,0.000000,0.226423,0.226423,0.0
2,0.316993,0.058496,0.058496,0.058496,0.058496,0.0,0.0,0.000000,0.058496,0.000000,0.000000,0.0


**2.word Embedding using BAG-OF-WORD (BOW)**

In [21]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
Text = """
The hotel was clean and the staff was very friendly.
I really enjoyed my stay there. The location was perfect.

"""
# TOKENIZATION
sentences = sent_tokenize(Text)
sentences = [sent.lower().replace(".","") for sent in sentences]
print('Our Corpus:',sentences)
#CountVectorizer : Convert a collection of text documents to a matrix of token counts.
count_vect = CountVectorizer()
# fit & transform will represent each sentences as BOW representation
BOW = count_vect.fit_transform(sentences)
# Get the vocabulary
print("Our vocabulary: ", count_vect.vocabulary_)
#see the BOW representation
print(f"BoW representation for {sentences[0]} {BOW[0].toarray()}")
print(f"BoW representation for {sentences[1]} {BOW[1].toarray()}")
print(f"BoW representation for {sentences[2]} {BOW[2].toarray()}")
# BOW representation for a new text
BOW_ = count_vect.transform(["Hotel review is nice"])
print("Bow representation for 'Hotel review is nice':", BOW_.toarray())


Our Corpus: ['\nthe hotel was clean and the staff was very friendly', 'i really enjoyed my stay there', 'the location was perfect']
Our vocabulary:  {'the': 11, 'hotel': 4, 'was': 14, 'clean': 1, 'and': 0, 'staff': 9, 'very': 13, 'friendly': 3, 'really': 8, 'enjoyed': 2, 'my': 6, 'stay': 10, 'there': 12, 'location': 5, 'perfect': 7}
BoW representation for 
the hotel was clean and the staff was very friendly [[1 1 0 1 1 0 0 0 0 1 0 2 0 1 2]]
BoW representation for i really enjoyed my stay there [[0 0 1 0 0 0 1 0 1 0 1 0 1 0 0]]
BoW representation for the location was perfect [[0 0 0 0 0 1 0 1 0 0 0 1 0 0 1]]
Bow representation for 'Hotel review is nice': [[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Word Embedding using n-Grams:**

In [22]:
from nltk import ngrams
text = input("Enter your sentence: ")
n = int(input("Enter the value of n: "))
n_grams = ngrams(text.split(),n)
for grams in n_grams:
  print(grams)


Enter your sentence: The quick brown fox jumps over the lazy dog
Enter the value of n: 3
('The', 'quick', 'brown')
('quick', 'brown', 'fox')
('brown', 'fox', 'jumps')
('fox', 'jumps', 'over')
('jumps', 'over', 'the')
('over', 'the', 'lazy')
('the', 'lazy', 'dog')
